In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-116836")
exp = Experiment(workspace=ws, name="quick-starts-ws-116836")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-116836
Azure region: southcentralus
Subscription id: a0f586ec-8016-4ea9-8248-9bf2299ad437
Resource group: aml-quickstarts-116836


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "azure-ml-cluster"
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already present.")
except ComputeTargetException:
    clus_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=0, max_nodes=4)

    cluster = ComputeTarget.create(ws, cluster_name, clus_config)
    cluster.wait_for_completion(show_output=True)



Cluster already present.


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.2,0.9),
    '--max_iter':choice(25,50, 75,100,125)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="training", compute_target=cluster_name, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='accuracy', estimator=est, policy=policy, primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4)

In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdriverun = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdriverun).show()
hyperdriverun.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cbfbfa48-a65f-498b-ad9b-66fb73e217a1
Web View: https://ml.azure.com/experiments/quick-starts-ws-116836/runs/HD_cbfbfa48-a65f-498b-ad9b-66fb73e217a1?wsid=/subscriptions/a0f586ec-8016-4ea9-8248-9bf2299ad437/resourcegroups/aml-quickstarts-116836/workspaces/quick-starts-ws-116836

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-09-30T11:03:59.590545][API][INFO]Experiment created<END>\n""<START>[2020-09-30T11:04:00.181102][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-09-30T11:04:00.677014][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-09-30T11:04:00.8513008Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_cbfbfa48-a65f-498b-ad9b-66fb73e217a1
Web View: https://ml.azure.com/experiments/quick-starts-ws-116836/runs/HD_cbfbfa48-a65f-498b-ad9b-66fb73e217a1?wsid=/subsc

{'runId': 'HD_cbfbfa48-a65f-498b-ad9b-66fb73e217a1',
 'target': 'azure-ml-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2020-09-30T11:03:59.075317Z',
 'endTimeUtc': '2020-09-30T11:11:31.181714Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/a0f586ec-8016-4ea9-8248-9bf2299ad437/resourceGroups/aml-quickstarts-116836/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-116836/experiments/**SCRUBBED**/runs/HD_cbfbfa48-a65f-498b-ad9b-66fb73e217a1_0, {\'error\': {\n    "additional_properties": {\n        "componentName": "execution-worker"\n    },\n    "error": {\n        "additional_properties": {\n            "referenceCode": null,\n            "messageFormat": null,\n            "messageParameters": null,\n            "severity": null\n        },\n        "code": "UserError",\n        "m

In [13]:
import joblib
# Get your best run and save the model from that run.
bestrun = hyperdriverun.get_best_run_by_primary_metric()
joblib.dump(bestrun, 'hyperdrive_bestrun_model.sav')


['hyperdrive_best_model.sav']

In [28]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dataset_link = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path=dataset_link).to_pandas_dataframe()

In [35]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=40)
total_train = pd.concat([x_train, y_train], axis=1)
total_test = pd.concat([x_test, y_test], axis=1)

       age          job  marital          education  default housing loan  \
0       57   technician  married        high.school       no      no  yes   
1       55      unknown  married            unknown  unknown     yes   no   
2       33  blue-collar  married           basic.9y       no      no   no   
3       36       admin.  married        high.school       no      no   no   
4       27    housemaid  married        high.school       no     yes   no   
...    ...          ...      ...                ...      ...     ...  ...   
32945   56    housemaid  married           basic.4y       no      no  yes   
32946   37   management  married  university.degree       no      no  yes   
32947   26       admin.   single  university.degree       no      no   no   
32948   31  blue-collar   single           basic.9y       no      no   no   
32949   39    housemaid  married           basic.4y       no      no   no   

         contact month day_of_week  ...  campaign  pdays  previous  \
0    

In [37]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=total_train,
    label_column_name='y',
    n_cross_validations=10)

In [39]:
# Submit your automl run

automl_run = exp.submit(automl_config,show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Running on local machine
Parent Run ID: AutoML_6088c302-3d33-41bc-a8e9-407dcf3aff16

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


In [ ]:
# Retrieve and save your best automl model.

bestrun_automl, fitted_automl =  automl.get_output()


In [ ]:
bestrun_automl.register_model(model_name='bestrun_automl.pkl', model_path='./outputs/')

In [ ]:
cluster.delete()
try:
    clsuter = ComputeTarget(workspace=ws, name= cluster_name)
    print('cluster deleted')
except:
    print("sorry could not delete")